In [1]:
import sys
sys.path.append('../src')
sys.path.append('../src/data/')
sys.path.append('../src/models/')
sys.path.append('../src/features/')
sys.path.append('../src/visualization/')
# will reload any library
%load_ext autoreload
%autoreload 2

In [2]:
# sportify specific
from data.metrics_track import r_precision, cosine_sim_closest, cosine_sim_top, NDCG
from visualization.plot_utils import write_latex_table
# general
import pandas as pd
import numpy as np
from tqdm import tqdm
from features.audio_features import AudioFeatures
from features.popularity_feature import Popularity

from features.tracks_info import TrackInfo
from collections import OrderedDict
import data.compressed_pickle as cpick
from visualization.plot_utils import glance_dict

# Load track data

In [18]:
cpick.save('../data/interim/popularity.pkl.bz2',result.to_dict()['popularity'])

Saving to ../data/interim/popularity.pkl.bz2


In [19]:
result = cpick.load('../data/interim/popularity.pkl.bz2')
glance_dict(result)

{'0UaMYEvWZi0ZqiDOoHU3YI': 66.0,
 '6I9VzXrHxO9rA9A5euc8Ak': 78.0,
 '0WqIKmW4BTrj3eJFmnCKMv': 60.0}

In [26]:
np.sum(list(result.values()))

34859818.39690124

In [25]:
mean_v = np.mean([v for v in list(result.values()) if not np.isnan(v) ])
for key,val in result.items():
    if np.isnan(val):
        result[key]=mean_v

In [3]:
tracks = TrackInfo(subset='5k-')


TypeError: preprocess() missing 1 required positional argument: 'df'

In [5]:
feat = Popularity()

In [7]:
pid=194
seed_t,seed_a, true_t,true_a,pool_t,pool_a = tracks.get_playlist_pooltest(pid=pid,k=25)

In [ ]:
tra

In [13]:
seed_f = feat.transform(seed_t)
pool_f = feat.transform(pool_t)

[[ 3.98241073]
 [ 3.12275737]
 [ 3.83913517]
 [ 1.33181287]
 [ 3.33767071]
 [ 1.18853731]
 [ 4.1256863 ]
 [ 3.62422183]
 [ 3.33767071]
 [ 4.34059964]
 [ 4.26896186]
 [ 3.83913517]
 [ 1.97655289]
 [ 4.34059964]
 [ 4.41223742]
 [ 3.83913517]
 [-1.03223387]
 [ 3.40930849]
 [ 3.62422183]
 [ 0.90198619]
 [ 3.55258405]
 [ 3.62422183]
 [ 4.4838752 ]
 [ 3.91077295]
 [ 3.69585961]]


In [6]:
k_range = [1, 5, 10, 25, 100]
max_picks = 500
# load playlist
pid = 194
pl_turi,pl_auri = tracks.get_playlist(pid)
n_songs = len(pl_turi)

In [17]:
suggest_df = pd.DataFrame()
n_top = 10
x_pred = OrderedDict()
stats = []
print('Test on playlist {:d}'.format(pid))
for k in k_range:
    x_pred[k] = []
    k_picks = int(max_picks / k)
    for i in range(k):
        x_query = xp[i - 1, :]
        ind, sim = cosine_sim_closest(X, x_query, k_picks)
        x_pred[k] = x_pred[k] + list(ind)
    subset_ids = pl_ints[k:]
    result = OrderedDict()
    result['k'] = k
    result['r precision'] = r_precision(subset_ids, x_pred[k])
    result['NDGC'] = NDCG(subset_ids, x_pred[k])
    stats.append(result)

sdf = pd.DataFrame(stats)
print(sdf)

write_latex_table(sdf, 'metrics', adir='.', render=True)

Test on playlist 194
     k  r precision      NDGC
0    1     0.023256  0.463649
1    5     0.024000  0.269681
2   10     0.033333  0.228421
3   25     0.009524  0.000000
4  100     0.000000  0.000000
